# Biconvex Spherical Lens

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchlensmaker as tlm

import math


surface = tlm.surfaces.Sphere(diameter=15, r=tlm.parameter(25))

optics = nn.Sequential(
    tlm.PointSourceAtInfinity(beam_diameter=18.5),
    tlm.Gap(10),
    tlm.RefractiveSurface(surface, n=(1.0, 1.5), anchors=("origin", "extent")),
    tlm.Gap(2),
    tlm.RefractiveSurface(surface, n=(1.5, 1.0), scale=-1, anchors=("extent", "origin")),
    tlm.Gap(30),
    tlm.FocalPoint(),
)

for name, p in optics.named_parameters():
    print(name, p)

tlm.show(optics, mode="2D", end=50)
tlm.show(optics, mode="3D", end=50)

In [ ]:
tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    sampling = {"dim": 2, "dtype": torch.float64, "base": 10},
    num_iter = 50
)

In [ ]:
tlm.show(optics, mode="2D", end=50)
tlm.show(optics, mode="3D", end=50)